In [ ]:
"""
Cell For Papermill Parameters
"""

ARCH = 'resnet50'
imagesize = 500
NUM_CLASSES = 2
saved_model_name = "model.pth.tar"
data_dir="../../dataset/ds1/val/"
file_name_out = "dnn_model_scores.csv"
wb_project = "DNN-Model-US-Inference"
GPU=0
SEED=1

In [ ]:
import os
import random
import shutil
import time
import pandas as pd
import numpy as np
import math
import json
import glob
from torch.utils.data.sampler import SubsetRandomSampler

from IPython.display import Image
import pydicom as dicom
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import csv
import math
import glob
from pathlib import Path
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import timm
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from datetime import timedelta
import datetime
import torch.distributed as dist

import wandb
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

import boto3
from botocore.exceptions import NoCredentialsError
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
%matplotlib inline

import torch.nn.functional as F
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
with open('../config/config.json') as config_f:
    data = json.load(config_f)
aws_access_key_id = data['aws_access_key_id']
aws_secret_access_key = data['aws_secret_access_key']
region_name = data['region_name']
WANDB_API_KEY = data['WANDB_API_KEY']

In [ ]:
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
wandb.login()
run = wandb.init(project=wb_project, entity='prostate-cancer', config={"imagesize":imagesize, "NUM_CLASSES":NUM_CLASSES})

wandb_run_name = wandb.run.name
wandb_run_id = wandb.run.id

config = wandb.config

In [ ]:
torch.cuda.set_device(GPU)

model = models.__dict__[ARCH](pretrained = True)
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)
model.cuda(GPU)

loc = 'cuda:{}'.format(GPU)
checkpoint = torch.load(saved_model_name, map_location=loc)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((imagesize,imagesize)),
    transforms.ToTensor()
])

In [ ]:
def predict_image(image):
    
    model.eval()
    with torch.no_grad():
    
        image_tensor = test_transforms(image).float()
        image_tensor = image_tensor.unsqueeze_(0)
        input = Variable(image_tensor)
        input = input.to(device)
        output = model(input)
        index = output.data.cpu().numpy().argmax()
        
        sm = torch.nn.Softmax()
        probabilities = sm(output) 
       
    return index,probabilities.tolist()[0]

In [ ]:
predicted = []
actual = []
image_name = []
probabilities = []

to_pil = transforms.ToPILImage()

fileList = glob.glob(data_dir + '*/*jpg*' , recursive=True)

In [ ]:
for image in fileList:
    img = cv2.imread(image)
    img = to_pil(img)
    prediction,prob = predict_image(img)
    predicted.append(prediction)
    probabilities.append(round(prob[0],4))
    image_name.append(image)
    
    if image.find('pos') > 0:
        actual.append(1)
    else:
        actual.append(0)    

In [ ]:
df = pd.DataFrame(list(zip(predicted, actual,probabilities,image_name)), columns =['predicted', 'actual','prob','image_name'])
df

In [ ]:
def add_patient_number(file_name,proc_type):
    """ Text """
    append = 'Prostate-MRI-US-Biopsy-' + file_name[-8:][:4]
    return append

In [ ]:
df['patient'] = df['image_name'].apply(add_patient_number,args=('stem',))

In [ ]:
df

In [ ]:
df.to_csv(file_name_out, sep=',', encoding='utf-8', index=False)

In [ ]:
def make_ROC_graph(labels_test,prediction):
    """ Text """

    false_positive_rate, recall, thresholds = roc_curve(labels_test,prediction)
    roc_auc = auc(false_positive_rate, recall)
    fig = plt.figure(figsize=(3, 3),frameon =False, dpi=200)  
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' %roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    fn = "roc_graph.png"
    plt.savefig(fn,bbox_inches='tight')
    plt.show()
    plt.close()
    wandb.log({"Media/ROC-Graph": wandb.Image("roc_graph.png")})

In [ ]:
make_ROC_graph(actual,predicted)